In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import numpy as np

<h1>Dataframe preparations</h1>

In [78]:
#import raw dataframe, adding genre and water lost

raw_diego_47 = pd.read_csv("/Users/rgu/Desktop/UROPs/UROP4/raw_diego_47.csv")
test = pd.read_csv("/Users/rgu/Desktop/UROPs/UROP4/raw_diego_47_2.csv")
#dry_weight is straight from the bag
#weight0 is after submerging in water for 1 minute
#weight1 is after 1 minute of the song
#weight2 is after another minute of the same song looped
raw_diego_47['song_id'] = raw_diego_47['song_id'].astype('category')
raw_diego_47['water_loss1'] = raw_diego_47['weight0'] - raw_diego_47['weight1']
raw_diego_47['water_loss2'] = raw_diego_47['weight1'] - raw_diego_47['weight2']
raw_diego_47['total_water_loss'] = raw_diego_47['weight0'] - raw_diego_47['weight2']
repeatedgenre = [val for val in test['genre'][0:47] for _ in range(2)]
raw_diego_47.insert(1, 'genre', repeatedgenre)
display(raw_diego_47)

,song_id,genre,speaker_id,dry_weight,weight0,weight1,weight2,water_loss1,water_loss2,total_water_loss
0,0,Hip Hop,1,1.1024,1.8595,1.8269,1.8176,0.0326,0.0093,0.0419
1,0,Hip Hop,2,1.4236,2.4471,2.3922,2.3757,0.0549,0.0165,0.0714
2,1,Hip Hop,1,1.3120,2.3055,2.2672,2.2519,0.0383,0.0153,0.0536
3,1,Hip Hop,2,1.3060,1.9372,1.9059,1.8930,0.0313,0.0129,0.0442
4,2,Hip Hop,1,1.1825,1.8883,1.8460,1.8392,0.0423,0.0068,0.0491
...,...,...,...,...,...,...,...,...,...,...
89,44,Rock,2,1.2036,2.2541,2.1999,2.1865,0.0542,0.0134,0.0676
90,45,Rock,1,1.3543,2.5484,2.5128,2.4917,0.0356,0.0211,0.0567
91,45,Rock,2,1.4669,2.5800,2.5387,2.5247,0.0413,0.0140,0.0553
92,46,Rock,1,1.0652,1.8188,1.8017,1.7909,0.0171,0.0108,0.0279


In [5]:
raw_diego_47.to_csv('/Users/rgu/Desktop/UROPs/UROP4/diego_47.csv', index = False)

In [105]:
#making a new dataframe that calculates the average weights and water loss
avg_diego_47 = pd.DataFrame()

avg_diego_47['song_id'] = raw_diego_47['song_id'].iloc[::2].values
avg_diego_47['avg_weight0'] = (raw_diego_47['weight0'][::2].values + raw_diego_47['weight0'][1::2].values)/2
avg_diego_47['avg_weight1'] = (raw_diego_47['weight1'][::2].values + raw_diego_47['weight1'][1::2].values)/2
avg_diego_47['avg_water_loss'] = avg_diego_47['avg_weight0'] - avg_diego_47['avg_weight1']


avg_diego_47['song_id'] = avg_diego_47['song_id'].astype('category')
display(avg_diego_47)

,song_id,avg_weight0,avg_weight1,avg_water_loss
0,0,2.15330,2.10955,0.04375
1,1,2.12135,2.08655,0.03480
2,2,1.91580,1.86810,0.04770
3,3,1.82995,1.77550,0.05445
4,4,2.44275,2.37980,0.06295
5,5,2.21200,2.16675,0.04525
6,6,2.43565,2.39155,0.04410
7,7,2.48585,2.44075,0.04510
8,8,3.27330,3.21245,0.06085
9,9,3.24150,3.19515,0.04635


In [106]:
avg_diego_47.to_csv("/Users/rgu/Desktop/UROPs/UROP4/avg_diego_47.csv", index=False)

In [6]:
#importing the fundamental frequency and RMS amplitude dataframe
audio_variables = pd.read_csv("/Users/rgu/Desktop/UROPs/UROP4/audio_variables.csv")
display(audio_variables)

,song_id,fundamental_frequency,RMS_amplitude
0,0,50.666667,11.059722
1,1,99.583403,18.580539
2,2,55.366667,5.501504
3,3,86.800000,7.562255
4,4,52.500000,9.823403
...,...,...,...
95,95,192.516667,11.155485
96,96,51.191865,7.901356
97,97,128.000000,16.168656
98,98,62.839527,13.849362


In [95]:
#adds fundamental frequency and RMS amplitude to the raw diego dataframe
repeatedfreq = [val for val in audio_variables['fundamental_frequency'] for _ in range(2)]
repeatedamp = [val for val in audio_variables['RMS_amplitude'] for _ in range(2)]


raw_diego_47['fundamental_frequency'] = repeatedfreq[0:94]
raw_diego_47['RMS_amplitude'] = repeatedamp[0:94]
display(raw_diego_47)

,song_id,genre,speaker_id,dry_weight,weight0,weight1,weight2,water_loss1,water_loss2,total_water_loss,...,Spectral_Contrast_2,Spectral_Contrast_3,Spectral_Contrast_4,Spectral_Contrast_5,Spectral_Contrast_6,Spectral_Contrast_7,Zero_crossing_rate,RMS,fundamental_frequency,RMS_amplitude
0,0,Hip Hop,1,1.1024,1.8595,1.8269,1.8176,0.0326,0.0093,0.0419,...,15.180902,19.460283,18.445573,17.748717,20.001675,48.807486,0.034923,0.298124,50.666667,11.059722
1,0,Hip Hop,2,1.4236,2.4471,2.3922,2.3757,0.0549,0.0165,0.0714,...,15.180902,19.460283,18.445573,17.748717,20.001675,48.807486,0.034923,0.298124,50.666667,11.059722
2,1,Hip Hop,1,1.3120,2.3055,2.2672,2.2519,0.0383,0.0153,0.0536,...,16.055847,18.854475,18.565686,18.523198,17.432226,48.269158,0.057866,0.254989,99.583403,18.580539
3,1,Hip Hop,2,1.3060,1.9372,1.9059,1.8930,0.0313,0.0129,0.0442,...,16.055847,18.854475,18.565686,18.523198,17.432226,48.269158,0.057866,0.254989,99.583403,18.580539
4,2,Hip Hop,1,1.1825,1.8883,1.8460,1.8392,0.0423,0.0068,0.0491,...,13.797597,16.896664,16.698401,18.815076,17.664029,47.102981,0.111268,0.307039,55.366667,5.501504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,44,Rock,2,1.2036,2.2541,2.1999,2.1865,0.0542,0.0134,0.0676,...,15.941027,18.918375,18.261114,18.521392,17.893833,47.778664,0.074939,0.348474,165.933333,10.393171
90,45,Rock,1,1.3543,2.5484,2.5128,2.4917,0.0356,0.0211,0.0567,...,13.447832,15.882746,15.777270,15.823381,15.771491,47.896708,0.086437,0.307516,73.100000,12.248029
91,45,Rock,2,1.4669,2.5800,2.5387,2.5247,0.0413,0.0140,0.0553,...,13.447832,15.882746,15.777270,15.823381,15.771491,47.896708,0.086437,0.307516,73.100000,12.248029
92,46,Rock,1,1.0652,1.8188,1.8017,1.7909,0.0171,0.0108,0.0279,...,14.796656,18.128270,18.525398,18.076727,17.678874,47.368980,0.115045,0.162226,82.747125,10.143955


<h1>One-Way ANOVA</h1>

In [23]:
#One-Way ANOVA for first minute. Dependent: song_id
model1 = ols('water_loss1 ~ C(song_id)', data=raw_diego_47).fit()

anova_results1 = sm.stats.anova_lm(model1, typ=2)
print(anova_results1)

#One-Way ANOVA for second minute. Dependent: song_id
model2 = ols('water_loss2 ~ C(song_id)', data=raw_diego_47).fit()

anova_results2 = sm.stats.anova_lm(model2, typ=2)
print(anova_results2)

              sum_sq    df         F    PR(>F)
C(song_id)  0.006672  46.0  1.101538  0.371082
Residual    0.006188  47.0       NaN       NaN
              sum_sq    df         F    PR(>F)
C(song_id)  0.001369  46.0  2.279693  0.002877
Residual    0.000614  47.0       NaN       NaN


In [78]:
#One-Way ANOVA for two minutes. Dependent: song_id
model_total = ols('total_water_loss ~ C(song_id)', data=raw_diego_47).fit()

anova_results_total = sm.stats.anova_lm(model_total, typ=2)
print(anova_results_total)

              sum_sq    df         F    PR(>F)
C(song_id)  0.008367  46.0  1.485053  0.090337
Residual    0.005757  47.0       NaN       NaN


In [98]:
#One-Way ANOVA for first minute. Dependent: genre
model1 = ols('water_loss1 ~ C(genre)', data=raw_diego_47).fit()

anova_results1 = sm.stats.anova_lm(model1, typ=2)
print(anova_results1)

#One-Way ANOVA for second minute. Dependent: genre
model2 = ols('water_loss2 ~ C(genre)', data=raw_diego_47).fit()

anova_results2 = sm.stats.anova_lm(model2, typ=2)
print(anova_results2)

            sum_sq    df         F   PR(>F)
C(genre)  0.001826   4.0  3.683102  0.00802
Residual  0.011033  89.0       NaN      NaN
            sum_sq    df        F   PR(>F)
C(genre)  0.000060   4.0  0.69336  0.59849
Residual  0.001923  89.0      NaN      NaN


In [99]:
#One-Way ANOVA for two minutes. Dependent: genre
model_total = ols('total_water_loss ~ C(genre)', data=raw_diego_47).fit()

anova_results_total = sm.stats.anova_lm(model_total, typ=2)
print(anova_results_total)

            sum_sq    df         F    PR(>F)
C(genre)  0.001643   4.0  2.928559  0.025177
Residual  0.012481  89.0       NaN       NaN


<h1>Adding Fundamental Frequency and RMS amplitude</h1>

In [97]:
def cutoffs(high_cutoff, mid_cutoff, lst):
    """
    categorizes a set of continuous data into categorical data of low, mid, high
    """
    categorized_list = []
    for i in lst:
            if i >= high_cutoff:
                categorized_list.append('high')
            elif i >= mid_cutoff:
                categorized_list.append('mid')
            else:
                categorized_list.append('low')
    return categorized_list

In [98]:
#categorizing the fundamental frequency and RMS amplitude by splitting into thirds (double check on this)

ff_base = min(raw_diego_47['fundamental_frequency'])
ff_range = max(raw_diego_47['fundamental_frequency']) - min(raw_diego_47['fundamental_frequency'])
ff_mid, ff_high = ff_range/3 + ff_base, 2*ff_range/3 + ff_base

rms_base = min(raw_diego_47['RMS_amplitude'])
rms_range = max(raw_diego_47['RMS_amplitude']) - min(raw_diego_47['RMS_amplitude'])
rms_mid, rms_high = rms_range/3 + rms_base, 2*rms_range/3 + rms_base

fund_freq_categorized = cutoffs(ff_high, ff_mid, raw_diego_47['fundamental_frequency'])
RMS_amp_categorized = cutoffs(rms_high, rms_mid, raw_diego_47['RMS_amplitude'])

In [80]:
#prints the cut offs for low, mid, high
print(ff_base, ff_mid, ff_high, ff_range)
print(rms_base, rms_mid, rms_high, rms_range)

0.0 97.42179252097573 194.84358504195146 292.2653775629272
2.56604584592661 8.176431431746416 13.78681701756622 16.831156757459414


In [28]:
#prints min max
print(min(raw_diego_47['fundamental_frequency']), max(raw_diego_47['fundamental_frequency']))
print(min(raw_diego_47['RMS_amplitude']), max(raw_diego_47['RMS_amplitude']))

0.0 292.2653775629272
2.56604584592661 19.397202603386024


In [69]:
def pie_counter(lst):
    """
    counts number of low, mid, high

    i mustve wrote this when really tired cause this is redundant as hell
    just do list.count() lol
    """
    low, mid, high = 0,0,0
    for i in lst:
        if i == 'low':
            low += 1
        if i == 'mid':
            mid += 1
        if i == 'high':
            high += 1
    return (low,mid,high,sum([low,mid,high]))

In [70]:
print(pie_counter(fund_freq_categorized)) #low, mid, high, total
print(pie_counter(RMS_amp_categorized))

(72, 18, 4, 94)
(20, 60, 14, 94)


In [90]:
#add categorized fundamental frequency and rms amplitude to the raw diego dataframe 
raw_diego_47['fund_freq_categorized'] = fund_freq_categorized
raw_diego_47['RMS_amp_categorized'] = RMS_amp_categorized

In [97]:
pd.set_option('display.max_rows', None)
display(raw_diego_47)


,song_id,genre,speaker_id,dry_weight,weight0,weight1,weight2,water_loss1,water_loss2,total_water_loss,fundamental_frequency,RMS_amplitude
0,0,Hip Hop,1,1.1024,1.8595,1.8269,1.8176,0.0326,0.0093,0.0419,50.666667,11.059722
1,0,Hip Hop,2,1.4236,2.4471,2.3922,2.3757,0.0549,0.0165,0.0714,50.666667,11.059722
2,1,Hip Hop,1,1.3120,2.3055,2.2672,2.2519,0.0383,0.0153,0.0536,99.583403,18.580539
3,1,Hip Hop,2,1.3060,1.9372,1.9059,1.8930,0.0313,0.0129,0.0442,99.583403,18.580539
4,2,Hip Hop,1,1.1825,1.8883,1.8460,1.8392,0.0423,0.0068,0.0491,55.366667,5.501504
5,2,Hip Hop,2,1.3074,1.9433,1.8902,1.8825,0.0531,0.0077,0.0608,55.366667,5.501504
6,3,Pop,1,1.1203,1.7845,1.7049,1.6962,0.0796,0.0087,0.0883,86.800000,7.562255
7,3,Pop,2,1.2198,1.8754,1.8461,1.8316,0.0293,0.0145,0.0438,86.800000,7.562255
8,4,Hip Hop,1,1.6164,2.4690,2.4086,2.4066,0.0604,0.0020,0.0624,52.500000,9.823403
9,4,Hip Hop,2,1.3161,2.4165,2.3510,2.3344,0.0655,0.0166,0.0821,52.500000,9.823403


<h1>Two-Way ANOVA</h1>

In [36]:
#two-way ANOVA for first minute
model = ols('water_loss1 ~ C(fund_freq_categorized) * C(RMS_amp_categorized)', data=raw_diego_47).fit()

anova_results3 = sm.stats.anova_lm(model, typ=2)
print(anova_results3)



                                                   sum_sq    df         F  \
C(fund_freq_categorized)                         0.000375   2.0  1.341283   
C(RMS_amp_categorized)                           0.000236   2.0  0.844710   
C(fund_freq_categorized):C(RMS_amp_categorized)  0.000223   4.0  0.399492   
Residual                                         0.012015  86.0       NaN   

                                                   PR(>F)  
C(fund_freq_categorized)                         0.266925  
C(RMS_amp_categorized)                           0.433216  
C(fund_freq_categorized):C(RMS_amp_categorized)  0.808508  
Residual                                              NaN  


In [33]:
#two-way ANOVA for second minute
model = ols('water_loss2 ~ C(fund_freq_categorized) * C(RMS_amp_categorized)', data=raw_diego_47).fit()

anova_results4 = sm.stats.anova_lm(model, typ=2)
print(anova_results4)

                                                   sum_sq    df         F  \
C(fund_freq_categorized)                         0.000067   2.0  1.557691   
C(RMS_amp_categorized)                           0.000022   2.0  0.501661   
C(fund_freq_categorized):C(RMS_amp_categorized)  0.000045   4.0  0.525305   
Residual                                         0.001853  86.0       NaN   

                                                   PR(>F)  
C(fund_freq_categorized)                         0.216505  
C(RMS_amp_categorized)                           0.607285  
C(fund_freq_categorized):C(RMS_amp_categorized)  0.717388  
Residual                                              NaN  


In [14]:
#test df
df = len(raw_diego_47) - len(raw_diego_47['song_id'].unique())
print(df)

47


<h1>Fisher's LSD</h1>

In [16]:
#Fisher's LSD
def fisherLSD(anova_table, n, alpha = 0.05):
    df = anova_table['df']['Residual']
    MSE = anova_table['sum_sq']['Residual']/df
    SE = (2*MSE/n) ** 0.5

    t = stats.t.ppf(1- alpha/2, df) #its like qt(0.975, df) in r

    return t*SE


In [39]:
fLSD_second_ids = fisherLSD(anova_results2, 2)
print(fLSD_second_ids) #LSD for 2nd minute of audio exposure

#with this value, compare it to the difference of two means. if the difference is greater or equal, the means are statistically different from each other. 

0.0072698865012020344


In [72]:
def compare_all_difs(names, means, lsd):
    total = []
    statistically_diff = []
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            if abs(means[i] - means[j]) >= lsd:
                statistically_diff.append((int(names[i]), int(names[j])))
            total.append((int(names[i]), int(names[j])))
    return total, statistically_diff

In [73]:
total, second_id_sig = compare_all_difs(avg_diego_47['song_id'], avg_diego_47['avg_water_loss'],fLSD_second_ids)
print(second_id_sig) #print all statistically significant mean pairs

print(len(second_id_sig), len(total)) #print number of statistically significant mean pairs, and the total number of pairs


[(0, 1), (0, 3), (0, 4), (0, 8), (0, 18), (0, 20), (0, 21), (0, 28), (0, 30), (0, 31), (0, 34), (0, 36), (0, 37), (0, 41), (0, 42), (0, 43), (0, 44), (0, 46), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 11), (1, 12), (1, 13), (1, 18), (1, 21), (1, 24), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 40), (1, 46), (2, 4), (2, 8), (2, 10), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 25), (2, 30), (2, 32), (2, 34), (2, 35), (2, 36), (2, 37), (2, 39), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (3, 4), (3, 5), (3, 6), (3, 7), (3, 9), (3, 10), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 19), (3, 20), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 32), (3, 33), (3, 34), (3, 35), (3, 36), (3, 37), (3, 38), (3, 39), (3, 40), (3, 41), (3, 42), (3, 43), (3, 44), (3, 45), (3, 46), (4, 5), (4, 6), (4, 7), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 19), (4, 2

<h1>ANCOVA</h1>

In [125]:
#ANCOVA testing. covariate: fundamental frequency
ancova_model = ols('total_water_loss ~ C(genre) + fundamental_frequency', data=raw_diego_47).fit() 

print(ancova_model.summary(), '\n')

print(sm.stats.anova_lm(ancova_model, typ=2))

                            OLS Regression Results                            
Dep. Variable:       total_water_loss   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     2.354
Date:                Wed, 03 Jul 2024   Prob (F-statistic):             0.0470
Time:                        19:06:22   Log-Likelihood:                 286.27
No. Observations:                  94   AIC:                            -560.5
Df Residuals:                      88   BIC:                            -545.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.05

In [77]:
iv_df = pd.read_csv('/Users/rgu/Desktop/UROPs/UROP4/pca_vars.csv')
del iv_df['Unnamed: 0']
display(iv_df)

,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,MFCC_10,...,Chroma_12,Spectral_Contrast_1,Spectral_Contrast_2,Spectral_Contrast_3,Spectral_Contrast_4,Spectral_Contrast_5,Spectral_Contrast_6,Spectral_Contrast_7,Zero_crossing_rate,RMS
0,-97.855042,135.672119,9.303637,43.661156,11.701764,8.463287,4.814926,4.632211,-2.996553,5.055413,...,0.354069,22.404477,15.180902,19.460283,18.445573,17.748717,20.001675,48.807486,0.034923,0.298124
1,-57.491714,100.315628,24.473669,19.702520,13.496424,18.411259,2.716806,7.863942,-1.652251,5.022759,...,0.283474,20.995998,16.055847,18.854475,18.565686,18.523198,17.432226,48.269158,0.057866,0.254989
2,-39.402782,65.279892,12.822299,22.813959,2.981905,7.124956,1.095100,5.395236,-2.655563,6.975082,...,0.386211,19.340803,13.797597,16.896664,16.698401,18.815076,17.664029,47.102981,0.111268,0.307039
3,-78.488525,93.797073,9.725464,22.979483,1.061313,9.407542,4.634785,15.640405,-13.076438,1.426162,...,0.422944,22.340985,14.924980,18.342329,17.555879,16.490619,17.438851,48.427292,0.055871,0.274789
4,-57.343479,57.145229,11.922997,18.735708,6.543367,7.913982,-5.598506,8.809514,-4.677073,7.306396,...,0.461535,21.166888,16.026787,18.047249,17.637104,18.720990,17.231853,48.264708,0.153140,0.157292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-10.877454,70.429535,13.961041,16.905106,-0.949582,10.882189,1.183557,10.941862,-1.581662,4.825974,...,0.450026,22.252014,15.840963,17.658659,17.450817,16.667889,15.325579,46.172651,0.092938,0.324382
96,-52.490955,79.570145,16.499453,13.872947,10.358987,-1.079784,4.116047,-2.320200,-0.904092,3.654129,...,0.303487,22.599723,15.904293,19.915363,19.354763,18.988615,19.611798,45.031942,0.092124,0.277571
97,-84.846443,110.007690,-10.243250,15.027181,-1.244845,10.339612,-4.690368,0.673799,-1.164039,0.960996,...,0.333793,18.903143,14.242540,17.767977,17.739449,19.186906,18.363384,47.798742,0.090422,0.181840
98,-23.682737,50.247078,2.391385,-2.780960,3.094078,3.128816,-12.155773,3.082805,-11.233156,6.026891,...,0.408483,20.189989,17.009887,19.769597,19.149256,20.461664,20.303627,48.987535,0.160864,0.287474


In [116]:
#adding the chroma 5 and spec contrast 4 IVs to the raw_diego_47 dataframe
repeatedchroma5 = [val for val in iv_df['Chroma_5'][0:47] for _ in range(2)]
repeatedspec4 = [val for val in iv_df['Spectral_Contrast_4'][0:47] for _ in range(2)]

raw_diego_47['Chroma_5'] = repeatedchroma5
raw_diego_47['Spectral_Contrast_4'] = repeatedspec4

display(raw_diego_47)


,song_id,genre,speaker_id,dry_weight,weight0,weight1,weight2,water_loss1,water_loss2,total_water_loss,...,Spectral_Contrast_2,Spectral_Contrast_3,Spectral_Contrast_4,Spectral_Contrast_5,Spectral_Contrast_6,Spectral_Contrast_7,Zero_crossing_rate,RMS,fundamental_frequency,RMS_amplitude
0,0,Hip Hop,1,1.1024,1.8595,1.8269,1.8176,0.0326,0.0093,0.0419,...,15.180902,19.460283,18.445573,17.748717,20.001675,48.807486,0.034923,0.298124,50.666667,11.059722
1,0,Hip Hop,2,1.4236,2.4471,2.3922,2.3757,0.0549,0.0165,0.0714,...,15.180902,19.460283,18.445573,17.748717,20.001675,48.807486,0.034923,0.298124,50.666667,11.059722
2,1,Hip Hop,1,1.3120,2.3055,2.2672,2.2519,0.0383,0.0153,0.0536,...,16.055847,18.854475,18.565686,18.523198,17.432226,48.269158,0.057866,0.254989,99.583403,18.580539
3,1,Hip Hop,2,1.3060,1.9372,1.9059,1.8930,0.0313,0.0129,0.0442,...,16.055847,18.854475,18.565686,18.523198,17.432226,48.269158,0.057866,0.254989,99.583403,18.580539
4,2,Hip Hop,1,1.1825,1.8883,1.8460,1.8392,0.0423,0.0068,0.0491,...,13.797597,16.896664,16.698401,18.815076,17.664029,47.102981,0.111268,0.307039,55.366667,5.501504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,44,Rock,2,1.2036,2.2541,2.1999,2.1865,0.0542,0.0134,0.0676,...,15.941027,18.918375,18.261114,18.521392,17.893833,47.778664,0.074939,0.348474,165.933333,10.393171
90,45,Rock,1,1.3543,2.5484,2.5128,2.4917,0.0356,0.0211,0.0567,...,13.447832,15.882746,15.777270,15.823381,15.771491,47.896708,0.086437,0.307516,73.100000,12.248029
91,45,Rock,2,1.4669,2.5800,2.5387,2.5247,0.0413,0.0140,0.0553,...,13.447832,15.882746,15.777270,15.823381,15.771491,47.896708,0.086437,0.307516,73.100000,12.248029
92,46,Rock,1,1.0652,1.8188,1.8017,1.7909,0.0171,0.0108,0.0279,...,14.796656,18.128270,18.525398,18.076727,17.678874,47.368980,0.115045,0.162226,82.747125,10.143955


In [117]:
#ANCOVA testing. Covariate: Chroma_5
ancova_model = ols('total_water_loss ~ C(genre) + Chroma_5', data=raw_diego_47).fit() 

print(ancova_model.summary())

print(sm.stats.anova_lm(ancova_model, typ=2))

                            OLS Regression Results                            
Dep. Variable:       total_water_loss   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     2.577
Date:                Wed, 03 Jul 2024   Prob (F-statistic):             0.0318
Time:                        18:46:23   Log-Likelihood:                 286.79
No. Observations:                  94   AIC:                            -561.6
Df Residuals:                      88   BIC:                            -546.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0638    

In [106]:
#ANCOVA testing. Covariate: Spectral_Contrast_4
ancova_model = ols('total_water_loss ~ C(genre) + Spectral_Contrast_4', data=raw_diego_47).fit() 

print(ancova_model.summary())

print(sm.stats.anova_lm(ancova_model, typ=2))

                            OLS Regression Results                            
Dep. Variable:       total_water_loss   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     2.970
Date:                Wed, 03 Jul 2024   Prob (F-statistic):             0.0159
Time:                        18:33:15   Log-Likelihood:                 287.70
No. Observations:                  94   AIC:                            -563.4
Df Residuals:                      88   BIC:                            -548.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0289    

<h1>Mediation</h1>

In [74]:
repeatediv = [val for val in iv_df['Chroma_1'][0:47] for _ in range(2)]

raw_diego_47['Chroma_1'] = repeatediv
display(raw_diego_47)


,song_id,genre,speaker_id,dry_weight,weight0,weight1,weight2,water_loss1,water_loss2,total_water_loss,MFCC_4,MFCC_5,MFCC_6,MFCC_7,MFCC_8,MFCC_9,MFCC_10,MFCC_11,MFCC_12,MFCC_13
0,0,Hip Hop,1,1.1024,1.8595,1.8269,1.8176,0.0326,0.0093,0.0419,43.661156,11.701764,8.463287,4.814926,4.632211,-2.996553,5.055413,11.722925,-1.356017,6.663229
1,0,Hip Hop,2,1.4236,2.4471,2.3922,2.3757,0.0549,0.0165,0.0714,43.661156,11.701764,8.463287,4.814926,4.632211,-2.996553,5.055413,11.722925,-1.356017,6.663229
2,1,Hip Hop,1,1.3120,2.3055,2.2672,2.2519,0.0383,0.0153,0.0536,19.702520,13.496424,18.411259,2.716806,7.863942,-1.652251,5.022759,-0.369152,5.798766,-3.976548
3,1,Hip Hop,2,1.3060,1.9372,1.9059,1.8930,0.0313,0.0129,0.0442,19.702520,13.496424,18.411259,2.716806,7.863942,-1.652251,5.022759,-0.369152,5.798766,-3.976548
4,2,Hip Hop,1,1.1825,1.8883,1.8460,1.8392,0.0423,0.0068,0.0491,22.813959,2.981905,7.124956,1.095100,5.395236,-2.655563,6.975082,0.770801,3.355919,-1.147394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,44,Rock,2,1.2036,2.2541,2.1999,2.1865,0.0542,0.0134,0.0676,21.551973,1.037404,3.704913,-9.348091,1.947141,-0.479380,8.561348,2.336440,9.005312,0.411698
90,45,Rock,1,1.3543,2.5484,2.5128,2.4917,0.0356,0.0211,0.0567,22.407558,0.481847,9.341730,-5.490591,3.233055,0.599401,10.460455,4.537220,8.770910,-1.652697
91,45,Rock,2,1.4669,2.5800,2.5387,2.5247,0.0413,0.0140,0.0553,22.407558,0.481847,9.341730,-5.490591,3.233055,0.599401,10.460455,4.537220,8.770910,-1.652697
92,46,Rock,1,1.0652,1.8188,1.8017,1.7909,0.0171,0.0108,0.0279,23.738234,-3.776206,7.043561,-6.969480,1.103500,-3.922772,-1.326794,-5.886769,0.863040,-7.163273


In [107]:
#Assess the effects of the IV on the DV
model_ivdv = ols('total_water_loss ~ C(genre)', data=raw_diego_47).fit() #simple linear regression model
print(model_ivdv.summary())

print(sm.stats.anova_lm(model_ivdv, typ=2))

                            OLS Regression Results                            
Dep. Variable:       total_water_loss   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     2.929
Date:                Wed, 03 Jul 2024   Prob (F-statistic):             0.0252
Time:                        18:36:20   Log-Likelihood:                 286.18
No. Observations:                  94   AIC:                            -562.4
Df Residuals:                      89   BIC:                            -549.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0591    

In [108]:
#Assess the effects of the IV on the Mediator
#testing fundamental frequency as the mediator
model_ivm = ols('MFCC_13 ~ C(genre)', data=raw_diego_47).fit() 
print(model_ivm.summary())

print(sm.stats.anova_lm(model_ivm, typ=2))

                            OLS Regression Results                            
Dep. Variable:                MFCC_13   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                    0.4627
Date:                Wed, 03 Jul 2024   Prob (F-statistic):              0.763
Time:                        18:36:28   Log-Likelihood:                -254.59
No. Observations:                  94   AIC:                             519.2
Df Residuals:                      89   BIC:                             531.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.8390    

In [109]:
#Assess the effects of M on DV, controlling for the IV
model_mdv = ols('total_water_loss ~ MFCC_13 + C(genre)', data=raw_diego_47).fit()
print(model_mdv.summary())

print(sm.stats.anova_lm(model_mdv, typ=2))

                            OLS Regression Results                            
Dep. Variable:       total_water_loss   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     2.383
Date:                Wed, 03 Jul 2024   Prob (F-statistic):             0.0447
Time:                        18:36:44   Log-Likelihood:                 286.34
No. Observations:                  94   AIC:                            -560.7
Df Residuals:                      88   BIC:                            -545.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0594    

In [122]:
def automated_testing(variable_frame, main_frame):
    for colname in variable_frame.columns:
        print('---------------------------')
        print(f'Current variable: {colname}')
        repeatediv = [val for val in iv_df[colname][0:47] for _ in range(2)]
        raw_diego_47[colname] = repeatediv

        model_ivm = ols(f'{colname} ~ C(genre)', data=main_frame).fit() 
        print('model: X on M \n')
        print('anova table \n', sm.stats.anova_lm(model_ivm, typ=2))
    
        
        model_mdv = ols(f'total_water_loss ~ {colname} + C(genre)', data=main_frame).fit()
        print('model: M on Y controlling for X \n')
        print('parameters \n', model_mdv.params)
        print('p-values \n', model_mdv.pvalues)
        print('anova table \n', sm.stats.anova_lm(model_mdv, typ=2))

        print('---------------------------')


In [123]:
automated_testing(iv_df, raw_diego_47)

---------------------------
Current variable: MFCC_1
model: X on M 

anova table 
                  sum_sq    df         F    PR(>F)
C(genre)   41964.498069   4.0  4.361972  0.002878
Residual  214056.866879  89.0       NaN       NaN
model: M on Y controlling for X 

parameters 
 Intercept              0.059963
C(genre)[T.Hip Hop]   -0.002604
C(genre)[T.Pop]       -0.002349
C(genre)[T.Rock]      -0.010674
C(genre)[T.Soul]      -0.014485
MFCC_1                 0.000011
dtype: float64
p-values 
 Intercept              8.159291e-22
C(genre)[T.Hip Hop]    5.696081e-01
C(genre)[T.Pop]        6.461174e-01
C(genre)[T.Rock]       4.503938e-02
C(genre)[T.Soul]       2.994842e-02
MFCC_1                 6.755128e-01
dtype: float64
anova table 
             sum_sq    df         F    PR(>F)
C(genre)  0.001615   4.0  2.853233  0.028288
MFCC_1    0.000025   1.0  0.176396  0.675513
Residual  0.012456  88.0       NaN       NaN
---------------------------
---------------------------
Current variable: MFC

<h1>Moderation</h1>

In [ ]:
model_moderation = ols('total_water_loss ~ genre * ', data=raw_diego_47).fit()

print(model_moderation.summary())